In [2]:
import pandas as pd

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171740 entries, 0 to 171739
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review_text  171739 non-null  object
 1   sentiment    171740 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.6+ MB


In [11]:
data = pd.read_csv("data/gpt_3.5_reviews.csv")

In [12]:
data.loc[:, 'review_text'] = data['review_text'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
data = data.loc[data["review_text"] != '']
data.loc[:, "sentiment"] = data["sentiment"].astype('int8')
data.reset_index(inplace=True, drop=True)

In [14]:
data["review_text"] = data["review_text"].astype(str)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['review_text'].values)
X = tokenizer.texts_to_sequences(data['review_text'].values)
X = pad_sequences(X)

In [16]:
Y = data['sentiment'].values

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [18]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.models import Sequential
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/home/ebozik/anaconda3/envs/nlp2/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-06-23 20:03:43.116243: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 20:03:43.261116: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 20:03:43.261158: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 20:03:43.263765: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file

In [21]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
batch_size = 32
epochs = 5

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
with tf.device('/CPU:0'):   
    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), callbacks=[early_stopping])
# model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test), callbacks=[early_stopping])

Epoch 1/5
4294/4294 ━━━━━━━━━━━━━━━━━━━━ 86s 20ms/step - accuracy: 0.9836 - loss: 0.0536 - val_accuracy: 0.9884 - val_loss: 0.0373
Epoch 2/5
4294/4294 ━━━━━━━━━━━━━━━━━━━━ 84s 20ms/step - accuracy: 0.9888 - loss: 0.0363 - val_accuracy: 0.9889 - val_loss: 0.0363
Epoch 3/5
4294/4294 ━━━━━━━━━━━━━━━━━━━━ 84s 20ms/step - accuracy: 0.9909 - loss: 0.0312 - val_accuracy: 0.9896 - val_loss: 0.0355
Epoch 4/5
4294/4294 ━━━━━━━━━━━━━━━━━━━━ 85s 20ms/step - accuracy: 0.9921 - loss: 0.0260 - val_accuracy: 0.9897 - val_loss: 0.0350
Epoch 5/5
4294/4294 ━━━━━━━━━━━━━━━━━━━━ 84s 20ms/step - accuracy: 0.9926 - loss: 0.0253 - val_accuracy: 0.9902 - val_loss: 0.0353
Restoring model weights from the end of the best epoch: 4.


In [22]:
import numpy as np
def predict_sentiment(text):
    text = text.lower().replace(r'[^\w\s]', '')
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=X.shape[1])
    pred = model.predict(padded)
    return 'Positive' if np.argmax(pred) == 1 else 'Negative'

In [23]:
texts = [
    "Akokoľvek sa na to pozriem tak to nevyzerá dobre.",
    "Tento deň je skvelý",
    "Neviem či som správne pochopil.",
    "To si vážne urobila?!",
    "To myslíš vážne?",
    "Ja som veľmi rád, že si sa to konečne naučila.",
    "Som pekný.",
    "Zajtra si kúpim pivo.",
    "Tak teraz neviem čo ďalej.",
    "Prehrali sme ale vidím na tom aj pozitívum.",
    "Tak som rozmýšlal, že to asi aj výjde."
]
for i in texts:
    print(i)
    print(predict_sentiment(i))

Akokoľvek sa na to pozriem tak to nevyzerá dobre.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
Negative
Tento deň je skvelý
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Positive
Neviem či som správne pochopil.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Negative
To si vážne urobila?!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Positive
To myslíš vážne?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Negative
Ja som veľmi rád, že si sa to konečne naučila.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Positive
Som pekný.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Positive
Zajtra si kúpim pivo.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Positive
Tak teraz neviem čo ďalej.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Positive
Prehrali sme ale vidím na tom aj pozitívum.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Positive
Tak som rozmýšlal, že to asi aj výjde.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Negative


In [23]:
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print("Test score:", score)
print("Test accuracy:", acc)

2/2 - 0s - 12ms/step - accuracy: 0.9639 - loss: 0.1018
Test score: 0.101809062063694
Test accuracy: 0.9639175534248352


In [24]:
model.save("lstm_gpt.keras")

In [25]:
import pickle
with open("lstm_gpt_tokenizer.pickle", "wb") as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
X.shape[1]

23